# Neural Network Implementation

In [ ]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

import lightgbm

from sklearn.externals import joblib
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import MinMaxScaler, LabelBinarizer, Imputer
from sklearn.utils import class_weight
#from category_encoders.hashing import HashingEncoder
from keras.layers import BatchNormalization, Dropout, Flatten, Dense
from keras.models import Sequential
from keras.layers import LeakyReLU
from keras.optimizers import Adam

from IPython.display import display

from utils import *

In [ ]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
%load_ext autoreload
%autoreload 2
%matplotlib inline

## Load Data

In [ ]:
train = pd.read_csv('input/train.csv')
print(train.shape)

In [ ]:
test = pd.read_csv('input/test.csv')
print(test.shape)

## Preprocessing

Removing service columns

In [ ]:
targets = train[['TARGET']]
train_size = train.shape[0]
train.drop(columns=['SK_ID_CURR', 'TARGET'], inplace=True)

In [ ]:
test_ids = test['SK_ID_CURR']
test = test.drop(columns=['SK_ID_CURR'])

In [ ]:
data = pd.concat([train, test])

Removing columns where too many missing values

In [ ]:
missing_vals = compute_missing_values(data)
to_remove = idx_of_missing_values(missing_vals)
data.drop(columns=to_remove, inplace=True)

Categorical Features

In [ ]:
data = pd.get_dummies(data)

Numerical Features Imputation

In [ ]:
numerical = data.select_dtypes('float64').columns.tolist()

In [ ]:
# median, so we do not that sensitive to outliers
imputer = Imputer(strategy = 'median')
# data[numerical] = imputer.fit_transform(data)
data[numerical] = imputer.fit_transform(data)

Features Normalization

In [ ]:
scaler = MinMaxScaler()
data[numerical] = scaler.fit_transform(data[numerical])

Computing class weights

In [ ]:
class_weights = class_weight.compute_class_weight('balanced', np.unique(targets), targets)
class_weights = dict(enumerate(class_weights))
class_weights

## Model

In [ ]:
model = Sequential()
model.add(Dense(512, input_shape=train.shape[1:]))
model.add(BatchNormalization())
model.add(LeakyReLU())

model.add(Dense(512))
model.add(BatchNormalization())
model.add(LeakyReLU())

model.add(Dense(512))
model.add(BatchNormalization())
model.add(LeakyReLU())

model.add(Dense(512))
model.add(BatchNormalization())
model.add(LeakyReLU())

model.add(Dense(512))
model.add(BatchNormalization())
model.add(LeakyReLU())

model.add(Dense(1, activation='sigmoid'))
model.summary()

In [ ]:
optimizer = Adam(lr=0.001)
model.compile(optimizer, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# model.fit(class_weights)